In [27]:
import sys
import polars as pl
import plotly.express as px
sys.path.insert(0, '..')
from fs_thesis import sql, show

# Create Overview for Tables Patients and Admissions

In [28]:
df = sql("SELECT * from hosp.patients")
show(df, limit=True)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaT
2,10000058,F,33,2168,2020 - 2022,NaT
3,10000068,F,19,2160,2008 - 2010,NaT
4,10000084,M,72,2160,2017 - 2019,2161-02-13
...,...,...,...,...,...,...
364622,19999828,F,46,2147,2017 - 2019,NaT
364623,19999829,F,28,2186,2008 - 2010,NaT
364624,19999840,M,58,2164,2008 - 2010,2164-09-17
364625,19999914,F,49,2158,2017 - 2019,NaT


In [29]:
df = sql("SELECT * from hosp.admissions")
show(df, limit=True)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P49AFC,TRANSFER FROM HOSPITAL,HOME,Medicaid,English,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P784FA,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P19UTS,EMERGENCY ROOM,HOSPICE,Medicaid,English,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P06OTX,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P39NWO,EMERGENCY ROOM,None,None,English,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546023,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P13JMH,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
546024,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,EW EMER.,P38XL8,PHYSICIAN REFERRAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
546025,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P33612,EMERGENCY ROOM,DIED,Private,English,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
546026,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,EW EMER.,P036NA,EMERGENCY ROOM,HOME,Private,English,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


# Patient Demographics & Time-to-Event Extraction

Extracts patient demographic data and calculates time-to-event metrics for survival analysis.

## Data Structure
- **Unique Identifier**: `subject_id`
- **Demographics**: gender, age, insurance, language, marital status, race
- **Baseline**: First admission timestamp (`t0_time`)
- **Event**: ICD-9/10 diagnosis code occurrence
- **Censoring**: Date of death (`dod`) or end of follow-up

## Output
Generates a dataset with:
- Demographic features
- Time to event (days from baseline)
- Event indicator (occurred/censored)
- Target classes: Early event (<1yr), late event (>1yr), or censored

In [30]:
df_baseline = sql("""SELECT 
    p.subject_id,
    p.gender,
    p.anchor_age,
    p.dod,
    -- Daten aus der ersten Aufnahme (Baseline)
    first_admit.admittime AS t0_time,
    first_admit.insurance,
    first_admit.language,
    first_admit.marital_status,
    first_admit.race,
    first_admit.admission_type
FROM hosp.patients p
INNER JOIN (
    -- Subquery, um nur die zeitlich erste Aufnahme pro Patient zu finden
    SELECT 
        subject_id, 
        admittime, 
        insurance, 
        language, 
        marital_status, 
        race, 
        admission_type,
        ROW_NUMBER() OVER (PARTITION BY subject_id ORDER BY admittime ASC) as row_num
    FROM hosp.admissions
) first_admit ON p.subject_id = first_admit.subject_id
WHERE first_admit.row_num = 1""")
show(df_baseline,limit=True)

,subject_id,gender,anchor_age,dod,t0_time,insurance,language,marital_status,race,admission_type
0,15231534,F,79,NaT,2128-12-29 03:16:00,Medicaid,Spanish,SINGLE,OTHER,EW EMER.
1,15232042,F,28,NaT,2126-12-26 23:17:00,Other,English,SINGLE,MULTIPLE RACE/ETHNICITY,EU OBSERVATION
2,15232385,F,84,NaT,2180-12-27 16:21:00,Medicare,English,SINGLE,WHITE,EW EMER.
3,15233654,F,32,NaT,2165-03-24 11:23:00,Private,English,MARRIED,HISPANIC OR LATINO,URGENT
4,15233681,F,30,NaT,2177-08-18 19:29:00,Private,English,SINGLE,OTHER,EU OBSERVATION
...,...,...,...,...,...,...,...,...,...,...
223447,13589246,M,22,NaT,2163-06-17 23:16:00,Private,English,SINGLE,WHITE,EW EMER.
223448,13590431,M,78,2164-08-22,2164-06-25 03:53:00,Medicare,English,MARRIED,WHITE,EW EMER.
223449,13591079,M,43,NaT,2111-01-01 22:08:00,Private,English,MARRIED,WHITE,DIRECT OBSERVATION
223450,13592954,M,62,NaT,2172-03-24 12:14:00,Private,English,MARRIED,WHITE,OBSERVATION ADMIT


In [31]:
# Zählt die eindeutigen Werte
anzahl = len(df["subject_id"].unique())
anzahl


223452

In [32]:
# Prüft auf Unique-Status
ist_unique = len(df["subject_id"].unique()) == len(df["subject_id"])
ist_unique

False

# Event Time

Extracts the first occurrence of the target diagnosis (ICD-50 code, Beispiel: Herzinsuffizienz) for each patient.

Returns:
- `subject_id`: Patient identifier
- `event_time`: Admission timestamp of first diagnosis occurrence

In [33]:
data = sql("""SELECT d.*, i.long_title
FROM hosp.diagnoses_icd d
LEFT JOIN hosp.d_icd_diagnoses i ON d.icd_code = i.icd_code
WHERE d.icd_code LIKE 'I50%'""")
show(data, limit=True)

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,10912602,23708919,8,I5032,10,Chronic diastolic (congestive) heart failure
1,10912800,24003151,2,I5032,10,Chronic diastolic (congestive) heart failure
2,10912800,24683894,3,I5032,10,Chronic diastolic (congestive) heart failure
3,10913302,20242491,5,I5022,10,Chronic systolic (congestive) heart failure
4,10913302,20317008,8,I5022,10,Chronic systolic (congestive) heart failure
...,...,...,...,...,...,...
43908,18804269,26989299,2,I5033,10,Acute on chronic diastolic (congestive) heart ...
43909,18804590,27623088,5,I5032,10,Chronic diastolic (congestive) heart failure
43910,18805396,25823725,9,I509,10,"Heart failure, unspecified"
43911,18806086,26366234,2,I5021,10,Acute systolic (congestive) heart failure


In [34]:
df_event = sql("""SELECT 
    d.subject_id,
    MIN(a.admittime) AS event_time
FROM hosp.diagnoses_icd d
JOIN hosp.admissions a ON d.hadm_id = a.hadm_id
WHERE d.icd_code LIKE 'I50%' 
GROUP BY d.subject_id
                         """)
show(df_event, limit=True)

,subject_id,event_time
0,11355925,2183-07-01 13:50:00
1,11364746,2165-10-29 21:07:00
2,11366422,2126-05-29 03:40:00
3,11367779,2131-03-02 16:59:00
4,11370994,2157-03-10 23:31:00
...,...,...
18885,17470420,2140-01-03 22:51:00
18886,17470897,2178-05-23 20:06:00
18887,17474083,2201-06-18 20:54:00
18888,17478746,2140-08-08 20:45:00


# Time-to-Event Calculation & Target Definition

Calculates time intervals and creates target classes based on event occurrence and timing.

## Steps:
1. **Join datasets**: Merges baseline demographics with event times (left join)
2. **Calculate time differences**: 
   - `t_event`: Days from baseline to diagnosis
   - `t_death`: Days from baseline to death
3. **Determine duration**: Uses event time if available, otherwise death time, fallback to 2000 days
4. **Handle errors**: Clips negative durations to 0
5. **Create target classes**:
   - Class 0: Early event (<365 days)
   - Class 1: Late event (>365 days)
   - Class 2: Censored (no event)

In [35]:
import polars as pl

df_final = (
    df_baseline.join(df_event, on="subject_id", how="left")
    # 1. Zeitdifferenzen und Event-Indikator berechnen
    .with_columns([
        ((pl.col("event_time") - pl.col("t0_time")).dt.total_days()).alias("t_event"),
        ((pl.col("dod") - pl.col("t0_time")).dt.total_days()).alias("t_death"),
        pl.col("event_time").is_not_null().cast(pl.Int32).alias("event_occurred")
    ])
    # 2. Duration festlegen (Priorität: Event > Tod > Fallback) und Clip
    .with_columns(
        pl.coalesce([
            pl.col("t_event"),
            pl.col("t_death"),
            pl.lit(2000)
        ])
        .clip(lower_bound=0)
        .alias("duration")
    )
    # 3. Zielvariable (Target) für TabPFN definieren
    .with_columns(
        pl.when((pl.col("event_occurred") == 1) & (pl.col("duration") <= 365))
        .then(0)     # Klasse 0: Früher Ausbruch (< 1 Jahr)
        .when((pl.col("event_occurred") == 1) & (pl.col("duration") > 365))
        .then(1)     # Klasse 1: Später Ausbruch (> 1 Jahr)
        .otherwise(2) # Klasse 2: Zensiert / Gesund / Kein Event
        .alias("target")
    )
)

# Ergebnis anzeigen
show(df_final, limit=True)

# Kurze Kontrolle der Verteilung
print(df_final["target"].value_counts())

,subject_id,gender,anchor_age,dod,t0_time,insurance,language,marital_status,race,admission_type,event_time,t_event,t_death,event_occurred,duration,target
0,15231534,F,79,NaT,2128-12-29 03:16:00,Medicaid,Spanish,SINGLE,OTHER,EW EMER.,NaT,NaN,NaN,0,2000,2
1,15232042,F,28,NaT,2126-12-26 23:17:00,Other,English,SINGLE,MULTIPLE RACE/ETHNICITY,EU OBSERVATION,NaT,NaN,NaN,0,2000,2
2,15232385,F,84,NaT,2180-12-27 16:21:00,Medicare,English,SINGLE,WHITE,EW EMER.,NaT,NaN,NaN,0,2000,2
3,15233654,F,32,NaT,2165-03-24 11:23:00,Private,English,MARRIED,HISPANIC OR LATINO,URGENT,NaT,NaN,NaN,0,2000,2
4,15233681,F,30,NaT,2177-08-18 19:29:00,Private,English,SINGLE,OTHER,EU OBSERVATION,NaT,NaN,NaN,0,2000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223447,13589246,M,22,NaT,2163-06-17 23:16:00,Private,English,SINGLE,WHITE,EW EMER.,NaT,NaN,NaN,0,2000,2
223448,13590431,M,78,2164-08-22,2164-06-25 03:53:00,Medicare,English,MARRIED,WHITE,EW EMER.,NaT,NaN,57.0,0,57,2
223449,13591079,M,43,NaT,2111-01-01 22:08:00,Private,English,MARRIED,WHITE,DIRECT OBSERVATION,NaT,NaN,NaN,0,2000,2
223450,13592954,M,62,NaT,2172-03-24 12:14:00,Private,English,MARRIED,WHITE,OBSERVATION ADMIT,NaT,NaN,NaN,0,2000,2


shape: (3, 2)
┌────────┬────────┐
│ target ┆ count  │
│ ---    ┆ ---    │
│ i32    ┆ u32    │
╞════════╪════════╡
│ 0      ┆ 10742  │
│ 2      ┆ 204562 │
│ 1      ┆ 8148   │
└────────┴────────┘


# Feature & Target Extraction

Prepares data for machine learning by selecting demographic features and target variable.

## Output:
- **X**: Feature matrix with demographics (pandas DataFrame)
  - gender, anchor_age, insurance, language, marital_status, race, admission_type
- **y**: Target vector with class labels (numpy array)
  - 0: Early event (<365 days)
  - 1: Late event (>365 days)
  - 2: Censored (no event)

In [ ]:
import polars as pl
from sklearn.model_selection import train_test_split

# 1. Konvertierung für Scikit-Learn (Split-Logik)
df_pd = df_final.to_pandas()

# Erster Split: Trenne das finale Test-Set ab (20% Hold-out)
df_train_val_raw, df_test = train_test_split(
    df_pd, 
    test_size=0.2, 
    stratify=df_pd["target"], 
    random_state=42
)

# Zweiter Split: Trenne den Rest in Training und Validierung (z.B. 80/20 vom Rest)
df_train_raw, df_val = train_test_split(
    df_train_val_raw, 
    test_size=0.2, 
    stratify=df_train_val_raw["target"], 
    random_state=42
)

# --- BACK TO POLARS: Balancing nur für das TRAINING ---
# Wir nehmen 3.000 Samples pro Klasse, um innerhalb des TabPFN-Limits zu bleiben
df_train_pl = pl.from_pandas(df_train_raw)
n_samples = 3000

df_balanced_train = pl.concat([
    df_train_pl.filter(pl.col("target") == 0).sample(n=min(n_samples, df_train_pl.filter(pl.col("target")==0).height), seed=42),
    df_train_pl.filter(pl.col("target") == 1).sample(n=min(n_samples, df_train_pl.filter(pl.col("target")==1).height), seed=42),
    df_train_pl.filter(pl.col("target") == 2).sample(n=n_samples, seed=42)
]).sample(fraction=1.0, shuffle=True, seed=42)

# --- FINALE FEATURES & TARGETS ---
feature_cols = ["gender", "anchor_age", "insurance", "language", "marital_status", "race", "admission_type"]

# Training (Balanced)
X_train = df_balanced_train.select(feature_cols).to_pandas()
y_train = df_balanced_train.select("target").to_series().to_numpy()

# Validation (Realistisch, nicht balanciert)
X_val = df_val[feature_cols]
y_val = df_val["target"].values

# Test (Realistisch, nicht balanciert - Hold-out)
X_test = df_test[feature_cols]
y_test = df_test["target"].values

print(f"Train (balanced): {len(y_train)} | Val (real): {len(y_val)} | Test (real): {len(y_test)}")

In [38]:
import polars as pl
from sklearn.model_selection import train_test_split

# 1. Konvertierung für Scikit-Learn (Split-Logik)
df_pd = df_final.to_pandas()

# Erster Split: Trenne das finale Test-Set ab (20% Hold-out)
df_train_val_raw, df_test = train_test_split(
    df_pd, 
    test_size=0.2, 
    stratify=df_pd["target"], 
    random_state=42
)

# Zweiter Split: Trenne den Rest in Training und Validierung (z.B. 80/20 vom Rest)
df_train_raw, df_val = train_test_split(
    df_train_val_raw, 
    test_size=0.2, 
    stratify=df_train_val_raw["target"], 
    random_state=42
)

# --- BACK TO POLARS: Balancing nur für das TRAINING ---
# Wir nehmen 3.000 Samples pro Klasse, um innerhalb des TabPFN-Limits zu bleiben
df_train_pl = pl.from_pandas(df_train_raw)
n_samples = 3000

df_balanced_train = pl.concat([
    df_train_pl.filter(pl.col("target") == 0).sample(n=min(n_samples, df_train_pl.filter(pl.col("target")==0).height), seed=42),
    df_train_pl.filter(pl.col("target") == 1).sample(n=min(n_samples, df_train_pl.filter(pl.col("target")==1).height), seed=42),
    df_train_pl.filter(pl.col("target") == 2).sample(n=n_samples, seed=42)
]).sample(fraction=1.0, shuffle=True, seed=42)

# --- FINALE FEATURES & TARGETS ---
feature_cols = ["gender", "anchor_age", "insurance", "language", "marital_status", "race", "admission_type"]

# Training (Balanced)
X_train = df_balanced_train.select(feature_cols).to_pandas()
y_train = df_balanced_train.select("target").to_series().to_numpy()

# Validation (Realistisch, nicht balanciert)
X_val = df_val[feature_cols]
y_val = df_val["target"].values

# Test (Realistisch, nicht balanciert - Hold-out)
X_test = df_test[feature_cols]
y_test = df_test["target"].values

print(f"Train (balanced): {len(y_train)} | Val (real): {len(y_val)} | Test (real): {len(y_test)}")

Train (balanced): 9000 | Val (real): 35753 | Test (real): 44691


> ℹ️ **INFO**  
> Why Train set is smaller than val and test. The Train set is balanced and gives a total of 9k for the TabPFN Algo. The Test and Val set is larger because it's a % of the total dataset (200k)